In [2]:
pip install implicit==0.7.2 --prefer-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=10797497 sha256=5394f82b4396781fb526db6558c39d5b03347ae764afed96408dba252736263b
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sp
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

from implicit.als import AlternatingLeastSquares

In [21]:
data = pd.read_csv("ecommerce_recommendation_dataset.csv")
df=pd.DataFrame(data)
print(df.shape)
print(df.columns)
df.head()

(60000, 51)
Index(['user_id', 'product_id', 'category', 'price', 'rating', 'review_count',
       'user_age', 'user_gender', 'user_location', 'purchase_history',
       'time_on_page', 'add_to_cart_count', 'search_keywords',
       'discount_applied', 'user_membership', 'user_browser', 'user_device',
       'purchase_time', 'session_duration', 'clicks_on_ads', 'page_views',
       'referral_source', 'wishlist_additions', 'cart_abandonment_rate',
       'average_spent', 'user_income', 'user_education', 'user_marital_status',
       'product_availability', 'stock_status', 'product_return_rate',
       'product_color', 'product_size', 'is_top_seller', 'discount_percentage',
       'time_to_purchase', 'delivery_time', 'shipping_fee', 'seller_rating',
       'seller_response_time', 'seller_location', 'product_rating_variance',
       'review_sentiment_score', 'user_engagement_score', 'ad_click_rate',
       'time_of_day', 'day_of_week', 'season', 'payment_method', 'coupon_used',
       'pro

,user_id,product_id,category,price,rating,review_count,user_age,user_gender,user_location,purchase_history,...,product_rating_variance,review_sentiment_score,user_engagement_score,ad_click_rate,time_of_day,day_of_week,season,payment_method,coupon_used,product_popularity
0,78517,1645,Books,842.23,2,155,24,Other,Urban,False,...,0.13,-0.28,0.68,0.04,Night,Thursday,Summer,Debit Card,False,0.54
1,52887,100,Books,253.76,3,331,43,Other,Suburban,False,...,0.02,0.28,0.11,0.89,Morning,Saturday,Summer,Debit Card,False,0.77
2,59395,585,Books,483.65,2,236,64,Female,Rural,True,...,1.55,0.23,0.35,0.99,Evening,Tuesday,Fall,Debit Card,False,0.14
3,54739,3774,Groceries,459.37,2,227,34,Female,Urban,False,...,1.41,0.93,0.73,0.16,Afternoon,Tuesday,Spring,Credit Card,False,0.18
4,42723,2119,Groceries,150.11,2,214,51,Female,Urban,True,...,1.29,0.11,0.26,0.17,Night,Wednesday,Spring,PayPal,False,0.66


In [22]:
cols_needed = [
    'user_id', 'product_id', 'purchase_history', 'category', 'price', 'discount_percentage',
    'is_top_seller', 'rating', 'review_count', 'product_popularity',
    'product_availability', 'stock_status', 'delivery_time', 'shipping_fee',
    'seller_rating', 'review_sentiment_score', 'product_color', 'product_size'
]

df = df[cols_needed].dropna(subset=['user_id', 'product_id'])

#Collaborative filtering

In [46]:

# Create user–item interaction matrix (implicit feedback)
train_df = (
    df.groupby('user_id', group_keys=False)
      .apply(lambda x: x.sample(frac=0.8, random_state=42))
      .reset_index(drop=True)
)
test_df = df.loc[~df.index.isin(train_df.index)].reset_index(drop=True)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df['interaction'] = 1
# --- 2️⃣ Create user–item interaction matrix (implicit feedback) ---
user_item_matrix = train_df.pivot_table(
    index='user_id',
    columns='product_id',
    values='interaction',
    aggfunc='sum',
    fill_value=0
)

# --- 3️⃣ Create mappings between real IDs and matrix indices ---
user_ids = list(user_item_matrix.index)
item_ids = list(user_item_matrix.columns)

user_mapping = {u: i for i, u in enumerate(user_ids)}
item_mapping = {i: j for j, i in enumerate(item_ids)}

# --- 4️⃣ Convert to sparse matrix ---
sparse_user_item = sp.csr_matrix(user_item_matrix.values)

# --- 5️⃣ Initialize and train ALS model ---
als_model = AlternatingLeastSquares(
    factors=50, regularization=0.1, iterations=20, random_state=42
)
als_model.fit(sparse_user_item)

print("ALS model trained. Matrix shape:", sparse_user_item.shape)

/tmp/ipython-input-3312635993.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=0.8, random_state=42))


Train shape: (57706, 18)
Test shape: (2294, 18)


  0%|          | 0/20 [00:00<?, ?it/s]

ALS model trained. Matrix shape: (45154, 4999)


#Content Based filtering

In [47]:
product_features = [
    'product_id', 'category', 'price', 'discount_percentage', 'is_top_seller',
    'rating', 'review_count', 'product_popularity', 'product_availability',
    'stock_status', 'delivery_time', 'shipping_fee', 'seller_rating',
    'review_sentiment_score', 'product_color', 'product_size'
]

product_df = df[product_features].drop_duplicates(subset=['product_id']).set_index('product_id')

# Encode categorical columns
availability_map = {
    'In Stock': 1,
    'Out of Stock': 0,
    'Pre-order': 0.5,
    'Low':0,
    'Medium':0.5,
    'High':1
}
product_df['product_availability'] = product_df['product_availability'].map(availability_map)
product_df['stock_status'] = product_df['stock_status'].map(availability_map)
cat_cols = ['category', 'product_color', 'product_size']
product_df = pd.get_dummies(product_df, columns=[c for c in cat_cols if c in product_df.columns], drop_first=True)

# Normalize numeric columns
scaler = MinMaxScaler()
product_df[product_df.columns] = scaler.fit_transform(product_df)



In [48]:
product_df.isnull().sum().sort_values(ascending=False).head(10)

,0
price,0
discount_percentage,0
is_top_seller,0
rating,0
review_count,0
product_popularity,0
product_availability,0
stock_status,0
delivery_time,0
shipping_fee,0


In [49]:
# Compute product similarity matrix
similarity_matrix = cosine_similarity(product_df)
similarity_df = pd.DataFrame(similarity_matrix, index=product_df.index, columns=product_df.index)

print("Product similarity matrix created:", similarity_df.shape)

Product similarity matrix created: (4999, 4999)


In [50]:
def get_cold_start_recommendations(top_n=5):
    cold_recs = (
        df.groupby('product_id')
        .agg({'is_top_seller': 'mean', 'discount_percentage': 'mean', 'purchase_history': 'mean'})
        .assign(score=lambda x: 0.6*x['is_top_seller'] + 0.4*x['discount_percentage'])
        .sort_values('score', ascending=False)
        .head(top_n)
        .index.tolist()
    )
    return cold_recs

In [53]:
def recommend_for_user(user_id, top_n=5, alpha=0.6):
    if user_id not in user_item_matrix.index:
        return get_cold_start_recommendations(top_n)

    user_idx = user_item_matrix.index.get_loc(user_id)


    if user_idx >= sparse_user_item.shape[0] or user_idx < 0:
        # This should theoretically not happen if user_mapping is correct
        # but is the final guard against out-of-bounds errors.
        print(f"DEBUG ALERT: Index {user_idx} is out of bounds for matrix size {sparse_user_item.shape[0]}. Falling back to cold start.")
        return get_cold_start_recommendations(top_n)

    user_interaction_row = sparse_user_item.getrow(user_idx)
    # --- CF scores ---
    cf_scores = {}
    recs = als_model.recommend(user_idx, user_interaction_row, N=top_n*5,filter_already_liked_items=True)
    for pid_idx, score in zip(*recs):
        pid = item_ids[pid_idx]
        cf_scores[pid] = score

    # --- Content-based scores ---
    liked_items = user_item_matrix.columns[user_item_matrix.iloc[user_idx] > 0]
    cb_scores = {}
    for pid in liked_items:
        if pid in similarity_df.index:
            sims = similarity_df[pid].sort_values(ascending=False)
            for similar_pid, sim_score in sims.items():
                cb_scores[similar_pid] = cb_scores.get(similar_pid, 0) + sim_score

    # Normalize both scores
    def normalize(d):
        if not d: return {}
        vals = np.array(list(d.values()))
        min_v, max_v = vals.min(), vals.max()
        return {k: (v-min_v)/(max_v-min_v+1e-9) for k, v in d.items()}

    cf_scores = normalize(cf_scores)
    cb_scores = normalize(cb_scores)

    # --- Hybrid combination ---
    final_scores = {}
    all_pids = set(cf_scores) | set(cb_scores)
    for pid in all_pids:
        final_scores[pid] = alpha * cf_scores.get(pid, 0) + (1-alpha) * cb_scores.get(pid, 0)

    ranked = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
    return [pid for pid, _ in ranked[:top_n]]

In [54]:
# Checking for known/repeated user
user_id = np.random.choice(user_ids)
user_index = user_mapping[user_id]
N=5

print(f"\nRecommendations for Known User {user_id}:")

# SAFETY CHECK: Ensure the index is within bounds (0 to N-1)
if user_index >= sparse_user_item.shape[0] or user_index < 0:
    print(f"Error: User index {user_index} out of bounds for matrix with {sparse_user_item.shape[0]} rows.")
    # Fallback to a valid index or skip
    user_index = 0 # Fallback example
    user_id_for_test = user_ids[user_index] # Update the user_id

# Get the user's interaction row (a 1xN sparse matrix)
user_interaction_row = sparse_user_item.getrow(user_index)
recommendations = als_model.recommend(user_index, user_interaction_row , N=N, filter_already_liked_items=True)

# Map ALS item indices back to actual product IDs
recommended_products = [item_ids[i] for i, score in zip(*recommendations)]
print(recommended_products)

# --- Cold-start case (user not in training data) ---
cold_user_id = 999999  # an unseen user
print(f"\nRecommendations for Cold-Start User {cold_user_id}:")

# Use your cold-start fallback (popularity or top sellers)
cold_recs = get_cold_start_recommendations(top_n=5)
print(cold_recs)


Recommendations for Known User 33667:
[3118, 2312, 4881, 4727, 2124]

Recommendations for Cold-Start User 999999:
[1333, 1931, 4917, 174, 4814]


In [55]:
def get_product_details(product_ids, df, top_n=5):
    """
    Fetch product details (name, category, price, rating, etc.)
    for the given list of product_ids.
    """
    rec_details = (
        df[df['product_id'].isin(product_ids)]
        [['product_id', 'category', 'price', 'rating']]
        .drop_duplicates(subset=['product_id'])
        .set_index('product_id')
        .reindex(product_ids)
    )
    return rec_details.head(top_n)

In [56]:
user_id = np.random.choice(user_item_matrix.index)

# Get recommendations
recommended_ids = recommend_for_user(user_id, top_n=5)
print(f"Recommendations for User {user_id}: {recommended_ids}")
# Get detailed info
rec_details = get_product_details(recommended_ids, df)
display(rec_details)

Recommendations for User 18183: [992, 4452, 4468, 102, 419]


,category,price,rating
product_id,,,
992,Electronics,193.28,2
4452,Clothing,558.22,1
4468,Books,252.48,5
102,Books,349.65,5
419,Groceries,767.50,4


In [57]:
def evaluate(alpha=0.6, top_n=5):
    results = {'precision': [], 'recall': [], 'map': [], 'ndcg': []}

    for user in tqdm(test_df['user_id'].unique()):
        actual_items = test_df[test_df['user_id'] == user]['product_id'].tolist()
        predicted_items = recommend_for_user(user, top_n=top_n, alpha=alpha)

        results['precision'].append(precision_at_k(actual_items, predicted_items, k=top_n))
        results['recall'].append(recall_at_k(actual_items, predicted_items, k=top_n))
        results['map'].append(average_precision_at_k(actual_items, predicted_items, k=top_n))
        results['ndcg'].append(ndcg_at_k(actual_items, predicted_items, k=top_n))

    print(f"\nAlpha = {alpha}")
    print(f"Precision@{top_n}: {np.mean(results['precision']):.4f}")
    print(f"Recall@{top_n}: {np.mean(results['recall']):.4f}")
    print(f"MAP@{top_n}: {np.mean(results['map']):.4f}")
    print(f"NDCG@{top_n}: {np.mean(results['ndcg']):.4f}")

    return results

In [58]:
def precision_at_k(actual, predicted, k=5):
    pred_k = predicted[:k]
    return len(set(pred_k) & set(actual)) / k if predicted else 0

def recall_at_k(actual, predicted, k=5):
    pred_k = predicted[:k]
    return len(set(pred_k) & set(actual)) / len(actual) if actual else 0

def average_precision_at_k(actual, predicted, k=5):
    score, hits = 0.0, 0.0
    for i, p in enumerate(predicted[:k]):
        if p in actual:
            hits += 1.0
            score += hits / (i + 1.0)
    return score / min(len(actual), k) if actual else 0.0

def ndcg_at_k(actual, predicted, k=5):
    def dcg(scores): return sum(s / np.log2(i + 2) for i, s in enumerate(scores))
    actual_set = set(actual)
    scores = [1 if p in actual_set else 0 for p in predicted[:k]]
    ideal = sorted(scores, reverse=True)
    return dcg(scores) / (dcg(ideal) + 1e-9)

In [59]:
results = {'precision': [], 'recall': [], 'map': [], 'ndcg': []}
for user in tqdm(test_df['user_id'].unique()):
    actual_items = test_df[test_df['user_id']==user]['product_id'].tolist()
    predicted_items = recommend_for_user(user, top_n=5, alpha=0.6)

    results['precision'].append(precision_at_k(actual_items, predicted_items))
    results['recall'].append(recall_at_k(actual_items, predicted_items))
    results['map'].append(average_precision_at_k(actual_items, predicted_items))
    results['ndcg'].append(ndcg_at_k(actual_items, predicted_items))

print(f"Precision@5: {np.mean(results['precision']):.4f}")
print(f"Recall@5: {np.mean(results['recall']):.4f}")
print(f"MAP@5: {np.mean(results['map']):.4f}")
print(f"NDCG@5: {np.mean(results['ndcg']):.4f}")

100%|██████████| 2272/2272 [00:27<00:00, 82.38it/s]

Precision@5: 0.0672
Recall@5: 0.3338
MAP@5: 0.1155
NDCG@5: 0.1696


In [60]:
for a in [0.2, 0.4, 0.6, 0.8]:
    print(f"\n--- Alpha = {a} ---")
    evaluate(alpha=a)


--- Alpha = 0.2 ---


100%|██████████| 2272/2272 [00:27<00:00, 81.98it/s]



Alpha = 0.2
Precision@5: 0.1343
Recall@5: 0.6690
MAP@5: 0.5912
NDCG@5: 0.6112

--- Alpha = 0.4 ---


100%|██████████| 2272/2272 [00:27<00:00, 82.24it/s]



Alpha = 0.4
Precision@5: 0.1235
Recall@5: 0.6153
MAP@5: 0.2995
NDCG@5: 0.3797

--- Alpha = 0.6 ---


100%|██████████| 2272/2272 [00:27<00:00, 82.26it/s]



Alpha = 0.6
Precision@5: 0.0672
Recall@5: 0.3338
MAP@5: 0.1155
NDCG@5: 0.1696

--- Alpha = 0.8 ---


100%|██████████| 2272/2272 [00:28<00:00, 79.06it/s]


Alpha = 0.8
Precision@5: 0.0174
Recall@5: 0.0871
MAP@5: 0.0292
NDCG@5: 0.0433


"The final model uses a 20/80 (CF/CB) blend. While this model is highly effective, the current high Recall@5 (0.6690) relative to Precision@5 (0.1343) indicates that the system is successfully identifying many relevant products but often ranks other, non-relevant items higher. The next steps will focus on improving the ranking quality of the CF model by introducing confidence weighting during the ALS fit, and refining the CB features to further boost Precision."